# INFO 4871 - Final Project
## Sentiment Analysis of Tweets Around Cryptocurrency
Mallory Hagen<br>
16 December 2019


For my final project, I will be using Twitter data to measure the sentiment of terms commonly used in reference to cryptocurrency (in particular, ‘bitcoin’). Using as streaming API, I collected tweets containing the term, 'bitcoin', over several days. Then, using the Python library Textblob I classified sentiment within these tweets to see if bitcoin generally skewed positive or negative, and if this sentiment could be corroborated with bitcoin stats.  

## Part One: Streaming API

Using the Python library <a href = 'https://www.tweepy.org/'>Tweepy</a> I created a basic stream listener. I filtered for tweets containing the word 'bitcoin'. The stream listener was run several times a day (typically covering morning, afternoon, and night) for several days. 

In [ ]:
import tweepy

api_key = 'x'
api_secret = 'x'
access_token = 'x'
access_secret = 'x'

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,data):
        with open('crypto_tweets.txt', 'a') as tweetFile:
            tweetFile.write(data)
            
    def on_error(self, status):
        print (status)

myListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myListener)

myStream.filter(track=['bitcoin'])
myStream.sample()

A little over 15000 tweets were collected over the course of a few days and saved to a txt file. 

In [65]:
import json

tweets_data = [] # empty list to hold the tweet data as json

tweets_file = open('crypto_tweets.txt', "r") # opening tweets file and assigning it to a variable to iterate through 

for line in tweets_file:
    
    try:  
        tweet = json.loads(line) # getting each tweet as json
        tweets_data.append(tweet)  # putting the tweets into the list
        
    except:  
        continue
    
print (tweet['text']) # looking at what current tweets look like

RT @qhfofficial: Our analysts made a video🎞 podcast about the general situation on the cryptocurrency market.
https://t.co/IwiekumcSH
#inve…


In [68]:
# Here's a simple DataFrame so we can quickly look at what kind of tweets we have from a glance.

import pandas as pd

tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))

tweets.head(10)

,text
0,Bitcoin’s “Supertrend” Flips Red as Price Fail...
1,RT @JacobCanfield: Is the price of Bitcoin goi...
2,Bitcoin’s “Supertrend” Flips Red as Price Fail...
3,RT @btcspeaks: 🔥5000 TRX up for grabs!🔥\n\n5 P...
4,@tylerwinklevoss @winklevoss Any comment? @rea...
5,"RT @ante_hero: 🤑Win 2,000 TRX as Credits🤑\n\n💥..."
6,RT @eminingsltd: 😍 Win #Giveaway 5000 Sats!\nR...
7,@carbonsaveruk @damian_from @MalcolmJEdwards @...
8,RT @LucasCosolo: 💸https://t.co/0A2BYjHHQa💸\nJo...
9,RT @harry_ugoji: Egoras Airdrop is now Live🚀💰🏆...


## Part Two: Preprocessing

Naturally, the tweets are fairly messy, with urls, emojis, random characters, etc. Before analyzing I want to clean these tweets to rid them of any messiness. This should be justified, since urls, emojis, etc. either can't be analyzed for sentiment or have no sentiment. We're mainly focused with individual words. 

In [74]:
import re # importing regular expression to clean up the tweets

clean_tweets = [] # list to hold the preprocessed tweets

for line in tweets_data:
    
    clean = re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", line['text']) # getting rid of urls and other nonessential characters
    clean_tweets.append(clean) # adding the preprocessed tweets into a new list
    
print(clean) # what the cleaned tweet looks like

RT qhfofficial Our analysts made a video podcast about the general situation on the cryptocurrency marketinve


## Part Three: Sentiment Analysis

After some deliberation, I decided to use the Python library <a href = 'https://textblob.readthedocs.io/en/dev/'>Textblob</a> to carry out my sentiment analysis. I focused on polarity (i.e. positive sentiment vs negative sentiment) rather than subjectivity (whether a tweet is objective or subjective).<br>
<br>
<a href = 'https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis'>Here</a> is the documentation for sentiment analysis using Textblob.

In [35]:
from textblob import TextBlob

In [117]:
sentiment_objects = [TextBlob(tweet) for tweet in clean_tweets] 
# Looping through the cleaned tweets using Textblob to do an analysis and storing them in a variable

sentiment_objects[0].polarity, sentiment_objects[0]
# Here's an example of Textblob getting the polarity from the first tweet in the dataset

(-0.25,
 TextBlob("Bitcoins Supertrend Flips Red as Price Fails to Surmount Low7000s gtgt   btc eth ltc xrp vet"))

In [77]:
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
# Getting the polarity values for each tweet in the Textblob

In [79]:
# Here is a rough DataFrame of the tweets and their polarity

sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])

sentiment_df.head(20)

,polarity,tweet
0,-0.250000,Bitcoins Supertrend Flips Red as Price Fails t...
1,0.000000,RT JacobCanfield Is the price of Bitcoin going...
2,-0.250000,Bitcoins Supertrend Flips Red as Price Fails t...
3,0.000000,RT btcspeaks 5000 TRX up for grabs5 People to ...
4,0.000000,tylerwinklevoss winklevoss Any comment realDon...
5,0.400000,RT antehero Win 2000 TRX as CreditsBankroll Ad...
6,0.800000,RT eminingsltd Win Giveaway 5000 SatsRandom 5...
7,0.000000,carbonsaveruk damianfrom MalcolmJEdwards RadRu...
8,0.000000,RT LucasCosolo Join now and start earning Bitc...
9,0.600000,RT harryugoji Egoras Airdrop is now LiveClick ...


In [122]:
# Here's another method of looking at the tweets and their polarity values, and whether that makes it
# positive, negative, or neutral

for tweet in clean_tweets:
    
    print(tweet)    
    analysis = TextBlob(tweet)    
    print(analysis.sentiment.polarity)    
    if analysis.sentiment[0]>0:       
        print ('Positive')   
    elif analysis.sentiment[0]<0:   
        print ('Negative')   
    else:       
        print ('Neutral')
    print('-----------------')


Bitcoins Supertrend Flips Red as Price Fails to Surmount Low7000s gtgt   btc eth ltc xrp vet
-0.25
Negative
-----------------
RT JacobCanfield Is the price of Bitcoin going up or is the purchasing power of the dollar going downIn 2011 1 could purchase 1 B
0.0
Neutral
-----------------
Bitcoins Supertrend Flips Red as Price Fails to SurmountLow7000s 
-0.25
Negative
-----------------
RT btcspeaks 5000 TRX up for grabs5 People to share 5000 TRXHow to participate1 Retweet and like this tweet2 Follow btcspea
0.0
Neutral
-----------------
tylerwinklevoss winklevoss Any comment realDonaldTrump knows 
0.0
Neutral
-----------------
RT antehero Win 2000 TRX as CreditsBankroll Advent Daily Giveaway Rules on poster Also Follow SkylimitBit Winner chosen S
0.4
Positive
-----------------
RT eminingsltd  Win Giveaway 5000 SatsRandom 5 followers will get 1000 Sats to each at 500 RetweetsTo Enter  Follow Retweet
0.8
Positive
-----------------
carbonsaveruk damianfrom MalcolmJEdwards RadRuminator jeremyc

0.2
Positive
-----------------
BITCOIN MAKE OR BREAK  FINALLY US LAW TO BOOST CRYPTO  JUSTIN BIEBER RUNS ON RIPPLE  ADA TRX  
0.0
Neutral
-----------------
Anyone else watch Queen of the South S3 E3  bit crypto scene in it shame its connected with coke dealers guess i 
0.0
Neutral
-----------------
RT alexlibertas NEW PODCASTBuilding Canadas Largest Bitcoin Mining CompanyBitcoinBroski of h4shr8 fame speaks with WesFulf
0.13636363636363635
Positive
-----------------
Bitcoins Supertrend Flips Red as Price Fails to Surmount Low7000s 
-0.25
Negative
-----------------
tylerwinklevoss winklevoss finkd do you have any comment to make realDonaldTrump and the white hats know It 
0.0
Neutral
-----------------
Hot off the press Crypto Church Daily News is out   artipatel CMDannCA bitcoin icymi
0.125
Positive
-----------------
RT CryptOwenWilson CHRISTMAS BITCOIN GIVEAWAY   Enter for your chance to win 00035 BTC   How to enter Like amp RT  F
0.8
Positive
-----------------
RT shairdropoff New givea

0.0
Neutral
-----------------
Over the past few weeks Bitcoin BTC has found itself consolidating in the low7000s holding this key suppor 
-0.15
Negative
-----------------
RT Bitsahara Get on the Lightning NetworkRETWEET FOR 2500 Satoshis and TAG a FriendFollow Bitsahara To WIN 8 WINNERS IN THE N
0.8
Positive
-----------------
RT tokenpay Where was the first place if at all that you spent crypto Need a last second gift idea Go to shopmoondick and order s
0.0375
Positive
-----------------
RT jsblokland More on bitcoin The number of addresses holding any amount of bitcoin has reached a new record taking out the number set
0.3181818181818182
Positive
-----------------
Bitcoins Supertrend Flips Red as Price Fails to Surmount Low7000s  
-0.25
Negative
-----------------
Current Bitcoin price trend prediction is WAIT ZONENext BTC price trend prediction is in 5 minPM us to j 
0.0
Neutral
-----------------
Top5 Cryptos This Week ATOM XTZ LINK BCH NEO bitcoin rossulbricht darknet silkroadmarketpl

TestTip Tip drawfairy by sending Bitcoin Cash BCH to bitcoincashqrelay2cafv5cl02c2rx4m6jvkjwfkhascr4gj7grz 
0.0
Neutral
-----------------
RT IvanOnTech FED pumping 425 BILLION of freshly printed fiat into the economy within 30 daysThis is 3x the entire Bitcoin market cap
0.15
Positive
-----------------
RT kingscrownBTC big ass interview with John McAfee that went live  READ NOW  we talk about bitmain rogerkver 
0.06818181818181818
Positive
-----------------
Bitcarla Youll find your Bitcoin 
0.0
Neutral
-----------------
 icoalert coin denver initialcoinofferings venture tokyo ICOs france tokenmarket 
0.0
Neutral
-----------------
Current Bitcoin price trend prediction is GOING UPNext BTC price trend prediction is in 5 minPM us to jo 
0.0
Neutral
-----------------
O valor do Bitcoin caiu   R2915096029
0.0
Neutral
-----------------
RT antehero Win 2000 TRX as BNKRBankroll Super Advent Daily Giveaway Rules on poster Also Follow ChoGriggs Winner chosen
0.37777777777777777
Positive
--------

-----------------
RT jsblokland ICYMI Millennials hold almost 2 of their pension savings in bitcoin data by Charles Schwab data shows 
0.0
Neutral
-----------------
DilemmaBeats RichardHeartWin CharlieShrem scottmelker Yet you buy BTC and what you wait for the establishmen 
0.0
Neutral
-----------------
RT socialstartnow1 NMXS ANNOUNCEDad wsj nytimes reuters bloomberg forbes nasdaq IHubStockPosts newyork business cnn bet f
0.0
Neutral
-----------------
Sales  Customer Service  Management to Upgrade  The Showalter Group    More Info    
0.5
Positive
-----------------
RT qhfofficial Our Fund successfullyheld its conference in Indonesiainvesting Topix CryptoNews trade t
0.0
Neutral
-----------------
Automated 8 Profit on XBT Why Automated Crypto Trading Is So Important  BitcoinForum 
0.4
Positive
-----------------
Melkam63983550 MoMwe Ethiopian not yet our Bank money market in the world We are on the way2 To purchase Bi 
0.0
Neutral
-----------------
RT LucasCosolo Join now and start earn

...

-----------------
if you wouldve told me 2 years ago that bitcoin was going to drop 85 in value alt coins would go down 99 id l 
-0.15555555555555559
Negative
-----------------
Hey check this outHeres What Burst Bitcoins 14000 Yearly Top and May Lead it Even Lower via Quarry app
0.5
Positive
-----------------
RT NexusOfficial Here is the first of a range of guides to help you use the NXS Wallet for purposes such as registering Assets and crea
0.125
Positive
-----------------
NatUnivHealthUS business Clearly lots of people I would tell you to spend your dollars on your bills and to save 
0.10000000000000002
Positive
-----------------
RT NexusOfficial Here is the first of a range of guides to help you use the NXS Wallet for purposes such as registering Assets and crea
0.125
Positive
-----------------
RT NexusOfficial Here is the first of a range of guides to help you use the NXS Wallet for purposes such as registering Assets and crea
0.125
Positive
-----------------
RT nanoevangelist Nan

0.25
Positive
-----------------
RT Rhythmtrader Well Bitcoin has stabilized at almost exactly 7000coin Im tired of waiting for a jump so Im taking the loss and g
-0.07500000000000001
Negative
-----------------
RT DACXio  The DACX Airdrop Campaign is live Participate and earn up to 75 DACX tokens for doing social tasks and 15 DACX tokens for
0.08484848484848484
Positive
-----------------
WordPress  PHP  Developer  Executive Resource Group Ltd    More Info      jobs 
0.5
Positive
-----------------
2019 telah terbukti sebagai tahun ketika teknologi blockchain turun ke bisnis Melangkah lebih jauh dari harihari 
0.0
Neutral
-----------------
RT NexusOfficial Here is the first of a range of guides to help you use the NXS Wallet for purposes such as registering Assets and crea
0.125
Positive
-----------------
Buy Bitcoin With PayPal Also with CC paysafecard Skrill OKPAY  btc bitcoin 20 
0.0
Neutral
-----------------
cryptocurrency bitcoin
0.0
Neutral
-----------------
RT TuurDemeester Searche

ETC Labs Core Rebrands to ETC Core to Clarify Difference With ETC Labs  Bitcoin 
0.0
Neutral
-----------------
Useful Tips To Protect Your Bitcoin From Hackers Some of the biggest things putting people off getting into Bitcoi 
0.3
Positive
-----------------
RT cryptochrisw December 2018 was the Bitcoin bottomDecember 2019 is the crypto twitter bottom
0.0
Neutral
-----------------
Bitcoin is completely neutral ground It offers the same functionality to   Krugman as to Satoshi It belongs to everyone even nocoiners
0.06666666666666667
Positive
-----------------
mrbitcorn ericjoo9 Bitcoin A trading challenge  How old are you If you want to act tough on Twitter go 
-0.14444444444444443
Negative
-----------------
Bitcoin no Atlas Quantum Corretoras Podem Quebrar a Qualquer Momento Decide Justia 
0.0
Neutral
-----------------
jimmysong Its a feature not a bugRecommend The Bitcoin Standard by Ammous
0.0
Neutral
-----------------
extstock QuraToken mantab gan juragancryptocurrency bitcoin
0.0
N

-----------------
RT CriptoNoticias  Llegaron las CriptoNoticias destacadas del da  16Dic Criptotendencias Argentina Venezuela Ethereum Bitcoin h
0.0
Neutral
-----------------
Wen you wake Bitcoin will be on Mars  
0.0
Neutral
-----------------
Ill say it Bitcoin will never be below 5900 going forward Currently 6900 Ive got my buys set in the 6000 
-0.25
Negative
-----------------
Cada de Bitcoin lleva al precio hacia USD 7000 Qusucedi  
0.0
Neutral
-----------------
derrick198S MonetaryWonk TheStalwart markets 
0.0
Neutral
-----------------
RT azorcode Bitcoinis eToroUS htcexodus Honestly understand Bitcoin after reading the book of Inventing Bitcoin skwp  and understa
0.6
Positive
-----------------
RT CriptoNoticias  Los abogados y despachos jurdicos ya pueden generar y gestionar contratos inteligentes en segundos a travs de la p
0.0
Neutral
-----------------
Bitcarla The real Bitcoin is at 
0.2
Positive
-----------------
RT DcntrlzdDan Bitcoinis eToroUS htcexodus 400000000th bitcoin

-----------------
70006800  via JpCointelegraph
0.0
Neutral
-----------------
RT CryptoNTez ETH  USD3D view 1 Year 2 UCTS signals Zooming in Looking for a 120 support retest as Bitcoin moves downETH
0.0
Neutral
-----------------
DINODDB Recommend using bitcoin for the free withdrawals Heres a referral link for a bonus if you sign up 
0.4
Positive
-----------------
RT bcbitcoin Join The BC Bitcoin Christmas Giveaway For The Chance To Win 2500 Tron TRXTo Enter Follow amp Like BC Bitcoin  Share amp
0.8
Positive
-----------------
RT RobertBeadles Yo100of BCH toSomeone  who answersI dont use stable coins for everything becausebitcoin crypto
0.0
Neutral
-----------------
wabi  btc 13 value 5 volume at Binance WaBi  Bitcoin wabiico cryptoasset 
0.0
Neutral
-----------------
Bitcoin really changed my life
0.2
Positive
-----------------
RT TheCryptoCandy 100 in Bitcoin BTC Giveaway2 RULES  Follow CryptoGuyUK amp Me Retweet amp Like this Tweet
0.0
Neutral
-----------------
RT cryptunez if you wo

For the purposes of this analysis: a polarity score below 0 is negative, above 0 is positive, and equal to 0 is neutral.

In [90]:
# Editing my previous DataFrame to include a tweets label (-1, 0, or 1), and whether that makes it
# positive (POS), negative (NEG), or neutral (NEU)

sentiment_df['label'] = 0
sentiment_df['sentiment'] = ""

sentiment_df.loc[sentiment_df['polarity'] > 0, 'label'] = 1
sentiment_df.loc[sentiment_df['polarity'] < 0, 'label'] = -1

sentiment_df.loc[sentiment_df['label'] > 0, 'sentiment'] = 'POS'
sentiment_df.loc[sentiment_df['label'] < 0, 'sentiment'] = 'NEG'
sentiment_df.loc[sentiment_df['label'] == 0, 'sentiment'] = 'NEU'

sentiment_df.head(20)

,polarity,tweet,label,sentiment
0,-0.250000,Bitcoins Supertrend Flips Red as Price Fails t...,-1,NEG
1,0.000000,RT JacobCanfield Is the price of Bitcoin going...,0,NEU
2,-0.250000,Bitcoins Supertrend Flips Red as Price Fails t...,-1,NEG
3,0.000000,RT btcspeaks 5000 TRX up for grabs5 People to ...,0,NEU
4,0.000000,tylerwinklevoss winklevoss Any comment realDon...,0,NEU
5,0.400000,RT antehero Win 2000 TRX as CreditsBankroll Ad...,1,POS
6,0.800000,RT eminingsltd Win Giveaway 5000 SatsRandom 5...,1,POS
7,0.000000,carbonsaveruk damianfrom MalcolmJEdwards RadRu...,0,NEU
8,0.000000,RT LucasCosolo Join now and start earning Bitc...,0,NEU
9,0.600000,RT harryugoji Egoras Airdrop is now LiveClick ...,1,POS


In [104]:
# Now we can see how many tweets fall into the three sentiment categories
print(sentiment_df.sentiment.value_counts())

# We can also look at what percent each sentiment makes up within my collected data
print(sentiment_df.sentiment.value_counts(normalize=True) * 100)

NEU    8040
POS    5635
NEG    1507
Name: sentiment, dtype: int64
NEU    52.957450
POS    37.116322
NEG     9.926228
Name: sentiment, dtype: float64
